In [1]:
import os
import json
vanilla, context = "/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_results/vanilla_o1-2024-12-17_Hi-ToM_data.json", "/data/jiarui_liu/social_reasoning_rl/social-world-model/data/hitom_results/socialized_context_o1-2024-12-17_processed_hitom_data.csv"

In [4]:
import pandas as pd
examples = [i for i in range(300)] + [i for i in range(600, 900)]

data, context_data = [], []
for i in examples:
    with open(os.path.join(vanilla, f"{i}.json"), "r") as f:
        data.append(json.load(f))
    with open(os.path.join(context, f"{i}.json"), "r") as f:
        context_data.append(json.load(f))
    
df1_prefixed = pd.DataFrame(data).add_prefix('vanilla_')
df2_prefixed = pd.DataFrame(context_data).add_prefix('context_')
df = pd.concat([df1_prefixed, df2_prefixed], axis=1)

# 

In [19]:
df['vanilla_question']

0                            Where is the tomato really?
1                         Where is the tangerine really?
2                        Where is the grapefruit really?
3                      Where is the green_pepper really?
4                            Where is the turnip really?
                             ...                        
595    Where does Jackson think Alexander thinks Nath...
596    Where does Amelia think Gracie thinks Ella thi...
597    Where does Logan think William thinks Sophia t...
598    Where does Chloe think Benjamin thinks Liam th...
599    Where does Aria think Lily thinks Mila thinks ...
Name: vanilla_question, Length: 600, dtype: object

# Error Analysis

In [6]:
df[['vanilla_is_correct', 'context_is_correct']].value_counts()

vanilla_is_correct  context_is_correct
True                True                  477
False               False                  48
True                False                  43
False               True                   32
Name: count, dtype: int64

In [7]:
tmp = df.loc[(df['vanilla_is_correct'] == False) & (df['context_is_correct'] == False)]

In [33]:
to_csv("hitom_vanilla_correct_context_incorrect.csv")

In [36]:
with open("hitom_vanilla_correct_context_incorrect.json", "w") as f:
    json.dump(tmp.to_dict(orient='records'), f, indent=2, ensure_ascii=False)

In [8]:
tmp.columns

Index(['vanilla_story', 'vanilla_question', 'vanilla_reasoning',
       'vanilla_answer', 'vanilla_correct_answer', 'vanilla_choices',
       'vanilla_is_correct', 'context_set_id', 'context_index',
       'context_deception', 'context_story_length', 'context_question_order',
       'context_story', 'context_question', 'context_reasoning',
       'context_answer', 'context_correct_answer',
       'context_socialized_context', 'context_extra_info', 'context_choices',
       'context_is_correct'],
      dtype='object')

In [9]:
id = 2

In [10]:
print(tmp.iloc[id]['context_story'])

1 Mila, Benjamin, Ethan, Isabella and Lily entered the porch.
2 The apple is in the blue_suitcase.
3 Mila made no movements and stayed in the porch for 1 minute.
4 Mila exited the porch.
5 Benjamin moved the apple to the green_bottle.
6 Benjamin exited the porch.
7 Ethan moved the apple to the blue_pantry.
8 Ethan exited the porch.
9 Isabella made no movements and stayed in the porch for 1 minute.
10 Isabella exited the porch.
11 Lily moved the apple to the blue_suitcase.
12 Lily exited the porch.
13 Mila, Benjamin, Ethan, Isabella and Lily entered the waiting_room.
14 Ethan, Lily and Benjamin entered the garden.
15 The beans is in the red_bathtub.
16 Ethan made no movements and stayed in the garden for 1 minute.
17 Ethan exited the garden.
18 Lily made no movements and stayed in the garden for 1 minute.
19 Lily exited the garden.
20 Benjamin made no movements and stayed in the garden for 1 minute.
21 Benjamin exited the garden.
22 Ethan, Lily and Benjamin entered the waiting_room.




In [11]:
tmp.iloc[id]['context_socialized_context']

{'agents_names': ['Mila', 'Benjamin', 'Ethan', 'Isabella', 'Lily'],
 'socialized_context': [{'timestep': '1',
   'state': 'none',
   'observations': {'Mila': 'none',
    'Benjamin': 'none',
    'Ethan': 'none',
    'Isabella': 'none',
    'Lily': 'none'},
   'actions': {'Mila': 'Mila entered the porch',
    'Benjamin': 'Benjamin entered the porch',
    'Ethan': 'Ethan entered the porch',
    'Isabella': 'Isabella entered the porch',
    'Lily': 'Lily entered the porch'}},
  {'timestep': '2',
   'state': 'Mila, Benjamin, Ethan, Isabella, and Lily are all in the porch. The apple is in the blue_suitcase.',
   'observations': {'Mila': '<same_as_state />',
    'Benjamin': '<same_as_state />',
    'Ethan': '<same_as_state />',
    'Isabella': '<same_as_state />',
    'Lily': '<same_as_state />'},
   'actions': {'Mila': 'none',
    'Benjamin': 'none',
    'Ethan': 'none',
    'Isabella': 'none',
    'Lily': 'none'}},
  {'timestep': '3',
   'state': 'Everyone remains in the porch with the appl

In [12]:
tmp.iloc[id]['context_question']

'Where does Mila think Lily thinks Benjamin thinks the apple is?'

In [13]:
print(tmp.iloc[id]['vanilla_reasoning'])

1) Initially, all five (Mila, Benjamin, Ethan, Isabella, Lily) see the apple in the blue_suitcase.  
2) Mila exits first (step 4), so she does not witness any later moves and believes the apple remains in the blue_suitcase.  
3) Benjamin leaves next (step 6), having moved the apple to the green_bottle (step 5). After he leaves, he does not see further moves, so he believes the apple is in the green_bottle.  
4) Ethan then moves the apple to the blue_pantry (step 7) before leaving (step 8).  
5) Lily eventually moves it again to the blue_suitcase (step 11) before leaving last (step 12).  
6) Lily knows Benjamin left right after placing the apple in the green_bottle, so from Lily’s point of view, Benjamin still thinks it is in the green_bottle.  
7) Mila, knowing Lily stayed long after Benjamin, will figure Lily saw Benjamin’s move to the green_bottle and knows Benjamin still thinks the apple is there.  
8) Therefore, “Where does Mila think Lily thinks Benjamin thinks the apple is?” = gr

In [14]:
print(tmp.iloc[id]['context_reasoning'])

Step-by-step reasoning:

1) Actual moves in the porch:  
   - After Mila leaves (she exited first), Benjamin moves the apple from the blue_suitcase → green_bottle, then exits.  
   - Next, Ethan moves it from green_bottle → blue_pantry, then exits.  
   - Lastly, Lily moves it from blue_pantry → blue_suitcase, then exits.

2) Benjamin’s own belief:  
   Since Benjamin left immediately after putting the apple in the green_bottle (and never saw Ethan or Lily’s later moves), Benjamin believes it is still in the green_bottle.

3) Lily’s belief about Benjamin’s belief:  
   Lily was present to see that Benjamin left right after placing it in green_bottle. Hence Lily thinks that Benjamin still believes it is in the green_bottle.

4) Mila’s belief about Lily’s belief about Benjamin:  
   Mila left before any moves, but she knows Benjamin exited after her and Lily exited last. Because Lily remained in the porch longer than Benjamin, Mila can deduce that Lily would realize Benjamin last saw the

In [15]:
print(tmp.iloc[id]['context_choices'])
print(tmp.iloc[id]['context_answer'], tmp.iloc[id]['context_correct_answer'])

A. red_bathtub, B. red_envelope, C. red_container, D. green_treasure_chest, E. red_drawer, F. red_cupboard, G. blue_suitcase, H. blue_pantry, I. green_envelope, J. green_bottle, K. green_container, L. red_treasure_chest, M. blue_basket, N. blue_drawer, O. red_crate
J blue_suitcase


In [16]:
tmp['context_question_order'].value_counts()

context_question_order
3    19
2    15
4    10
1     4
Name: count, dtype: int64

In [17]:
tmp['context_deception'].value_counts()

context_deception
True     43
False     5
Name: count, dtype: int64

In [18]:
tmp['context_story_length'].value_counts()

context_story_length
3    24
1    13
2    11
Name: count, dtype: int64